<a href="https://colab.research.google.com/github/CagataySencan/Hybrid-Movie-Recommender-System/blob/main/HybridMovieRecommanderEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genel Hazırlıklar

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
!pip install scikit-surprise
from surprise import Reader,Dataset,SVD

     |████████████████████████████████| 11.8 MB 136 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630120 sha256=f19c38c36494c808d2cb97ad8ae99c67badde78409c46a989bfbaec8f0104175
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
! pip install kaggle

In [3]:
! mkdir ~/.kaggle

In [4]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download rounakbanik/the-movies-dataset
! kaggle datasets download tmdb/tmdb-movie-metadata

 98% 224M/228M [00:02<00:00, 79.2MB/s]
100% 228M/228M [00:02<00:00, 81.5MB/s]
 90% 8.00M/8.89M [00:00<00:00, 9.32MB/s]
100% 8.89M/8.89M [00:00<00:00, 9.53MB/s]


In [7]:
! unzip /content/the-movies-dataset.zip
! unzip /content/tmdb-movie-metadata.zip

Archive:  /content/the-movies-dataset.zip
  inflating: credits.csv             
  inflating: keywords.csv            
  inflating: links.csv               
  inflating: links_small.csv         
  inflating: movies_metadata.csv     
  inflating: ratings.csv             
  inflating: ratings_small.csv       
Archive:  /content/tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [8]:
credits = pd.DataFrame(pd.read_csv('credits.csv'))
keywords = pd.DataFrame(pd.read_csv('keywords.csv'))
movies_metadata_tmdb = pd.DataFrame(pd.read_csv('tmdb_5000_movies.csv'))
movies_tmdb_credits = pd.DataFrame(pd.read_csv('tmdb_5000_credits.csv'))
movies_metadata = pd.DataFrame(pd.read_csv('movies_metadata.csv'))
ratings = pd.DataFrame(pd.read_csv('ratings.csv'))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Content Based Filtering


In [9]:
# Content based filtering için hazırlık

content_based_movies = movies_metadata_tmdb[['overview','title','genres', 'keywords']]
content_based_credits = movies_tmdb_credits[['cast','crew','movie_id']]
content_based = content_based_movies.join(content_based_credits)
# Yeterli veri olduğu için eksik veri olan satırları çıkartma kararı aldım
content_based = content_based.dropna()

# Bu filtreden en yüksek verimi alabilmek için iki alt filtreye bölme kararı aldım
# İlk filtre önerileri overview'a, ikincisi ise cast, crew, keyword, genre gibi parametrelere göre öneri yapacak
# Bu filtrede memory yetersizliği nedeniyle cosine similarity score hesaplayamadığım için TMDB 5000 Movie Dataset'i kullandım
content_based.head()

,overview,title,genres,keywords,cast,crew,movie_id
0,"In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",19995
1,"Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",285
2,A cryptic message from Bond’s past sends him o...,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",206647
3,Following the death of District Attorney Harve...,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",49026
4,"John Carter is a war-weary, former military ca...",John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",49529


### Overview Bazlı Filtre

In [10]:
# overview bazlı filtre :

overview_based = content_based[['title','overview']]
print(overview_based.isnull().sum())
print(overview_based.head())

# Vektörleştirme işlemi 

tf_idf = TfidfVectorizer(stop_words='english')
tfidf_matrix_overview = tf_idf.fit_transform(overview_based['overview'])


# Filmler arasındaki benzerlikleri nümerik bir skorla görmek için 'cosine similarity score' hesabı yapma kararı aldım
tfidf_matrix_overview.shape
similarity_overview = linear_kernel(tfidf_matrix_overview,tfidf_matrix_overview)
overview_based = overview_based.reset_index()
index = pd.Series(overview_based.index, index=overview_based['title']).drop_duplicates()

title       0
overview    0
dtype: int64
                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  
0  In the 22nd century, a paraplegic Marine is di...  
1  Captain Barbossa, long believed to be dead, ha...  
2  A cryptic message from Bond’s past sends him o...  
3  Following the death of District Attorney Harve...  
4  John Carter is a war-weary, former military ca...  


In [11]:
# Girilen film ismine göre öneri yapacak olan fonksiyon :

def recommend_by_overview(title, sim = similarity_overview) :
  idx = index[title]
  sim_score = list(enumerate(sim[idx]))
  sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)
  sim_score = sim_score[1:11]
  movie_index = [i[0] for i in sim_score]
  recommendation = overview_based['title'].iloc[movie_index].tolist()
  recDict = {}
  i = 0
  while i < 10 :
    recDict[recommendation[i]] = sim_score[i][1]
    i += 1

  return recDict

### Cast, Crew, Keyword, Genre Bazlı Filtre

In [12]:
features = ['cast', 'crew', 'keywords', 'genres']
feature_based = content_based[['title','cast', 'crew', 'keywords', 'genres']]

for feature in features : 
  feature_based[feature] = feature_based[feature].apply(literal_eval)

# Yönetmenleri crew sütunundan almak için gerekli fonksiyon  
def add_director(j) :
  for i in j :
    if i['job'] == 'Director':
      return i['name']
  return np.nan

# Diğer sütunlardaki bilgilerden ilk 5 tanesini almak için gerekli fonksiyon
def add_list(j) :
  if isinstance(j, list):
        
        names = [i['name'] for i in j]
        
        if len(names) > 5:
            names = names[:5]
        return names
   
  return []       

# Veriyi kullanabilmek için uyumlu forma getirme işlemi
feature_based['director'] = feature_based['crew'].apply(add_director)
features = ['cast', 'keywords', 'genres']

for feature in features:
    feature_based[feature] = feature_based[feature].apply(add_list)

feature_based = feature_based.drop(columns = ['crew'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [13]:
# Bütun satırlardaki verileri küçük harfe dönüştürme ve boşlukları düzenleme işlemi 
def clean(j):
    if isinstance(j, list):
        return [str.lower(i.replace(" ", "")) for i in j]
    else:
        if isinstance(j, str):
            return str.lower(j.replace(" ", ""))
        else:
            return ''

features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    feature_based[feature] = feature_based[feature].apply(clean)

# Vektörleştirme işlemini tek seferde yapabilmek için bütün verileri içeren tek bir sütun oluşturma işlemi
def all_data(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])


feature_based['all'] = feature_based.apply(all_data, axis=1)

tf_idf = TfidfVectorizer(stop_words='english')
tfidf_matrix_feature = tf_idf.fit_transform(feature_based['all'])

# Filmler arasındaki benzerlikleri nümerik bir skorla görmek için 'cosine similarity score' hesabı yapma kararı aldım
tfidf_matrix_feature.shape
similarity_feature = linear_kernel(tfidf_matrix_feature,tfidf_matrix_feature)
feature_based = feature_based.reset_index()
index = pd.Series(feature_based.index, index=feature_based['title']).drop_duplicates()

In [14]:
def recommend_by_feature(title, sim = similarity_feature) :
  idx = index[title]
  sim_score = list(enumerate(sim[idx]))
  sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)
  sim_score = sim_score[1:11]
  movie_index = [i[0] for i in sim_score]
  recommendation = feature_based['title'].iloc[movie_index].tolist()
  recDict = {}
  i = 0
  while i < 10 :
    recDict[recommendation[i]] = sim_score[i][1]
    i += 1

  return recDict

In [ ]:
# İki filtreden de gelen veriler dictionary şeklinde olduğundan dolayı değerden anahtar kelimeyi bulan bir fonksiyona ihtiyacım oldu
def get_key(d, val):
    keys = [k for k, v in d.items() if v == val]
    if keys:
        return keys[0]
    return None

# Buradaki fonksiyonda iki filtreden gelen cosine similarity score değeri bir listeye atanıyor 
# Liste büyükten küçüğe sıralanıp değeri en büyük olan 10 film öneri olarak veriliyor
def final_recommendation_content_based(title) :
  list1 = recommend_by_overview(title)
  list2 = recommend_by_feature(title)
  value_list = list(list1.values()) + list(list2.values())
  value_list.sort(reverse = True)
  recommend_list = []
  i = 0 
  while i < 20 :
    key = get_key(list1,value_list[i])
    if key is None :
      new_key = get_key(list2,value_list[i])
      if new_key in recommend_list :
        i += 1 
        continue
      else :
        recommend_list.append(new_key)    
    else :
      if key in recommend_list :
        i += 1 
        continue
      else :
        recommend_list.append(key)  
    i += 1  
  recommend_list = recommend_list[0:10]
  return recommend_list

final_recommendation_content_based('Bruce Almighty')

# Collaborative Filtering